<a href="https://colab.research.google.com/github/thpak1983/updated/blob/main/Most_update_autotrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 1 秒
        time.sleep(1.0)

        # 每 10 個請求後額外等待
        if request_count % 10 == 0:
            time.sleep(1)  # 額外等待 1 秒

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 檢查今日訂單
        try:
            today_orders = trade_ctx.today_orders(status=[OrderStatus.PartialFilled, OrderStatus.New])
            stocks_with_open_orders = set(order.symbol for order in today_orders)
            logger.info(f"{current_date}: 今日未完成訂單的股票: {stocks_with_open_orders}")

            if symbol in stocks_with_open_orders:
                logger.info(f"{current_date}: {symbol} 已有未完成訂單，跳過交易")
                return
        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                position_found = False
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            quantity = position.quantity
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            logger.info(f"{current_date}: {symbol} 當前持倉: 數量 {quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            if current_price <= stop_loss_price:
                                logger.info(f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            if current_price >= take_profit_price:
                                logger.info(f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 買入和賣出邏輯
        if latest_row['buy_signal']:
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif latest_row['sell_signal'] and (current_time - latest_row['timestamp']).total_seconds() < 300:  # 5分鐘內
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'] > df['D']) & (df['K'].shift(1) <= df['D'].shift(1))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'] > 30) & (df['RSI'].shift(1) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'] < df['SMA10']) & (df['SMA10'] < df['SMA20']) & (df['SMA20'] < df['SMA50'])

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'] > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系統啟動")
    while True:
        try:
            current_time = datetime.now()

            # 每次循環開始時更新數據
            update_data()

            # 檢查是否在交易時間內
            if current_time.weekday() < 5 and 9 <= current_time.hour < 16:
                logger.info(f"檢查時間: {current_time}")
                for stock in global_STOCK_LIST:
                    logger.info(f"檢查股票: {stock}")
                    check_and_trade(stock)

            # 等待到下一分鐘開始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循環中發生錯誤: {e}", exc_info=True)
            time.sleep(60)  # 發生錯誤時等待1分鐘後繼續

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等

ModuleNotFoundError: No module named 'longport'

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 1 秒
        time.sleep(1.0)

        # 每 10 個請求後額外等待
        if request_count % 10 == 0:
            time.sleep(1)  # 額外等待 1 秒

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 檢查今日訂單
        try:
            today_orders = trade_ctx.today_orders(status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew])
            stocks_with_open_orders = set(order.symbol for order in today_orders)
            logger.info(f"{current_date}: 今日未完成訂單的股票: {stocks_with_open_orders}")

            if symbol in stocks_with_open_orders:
                logger.info(f"{current_date}: {symbol} 已有未完成訂單，跳過交易")
                return
        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                position_found = False
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            quantity = position.quantity
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            logger.info(f"{current_date}: {symbol} 當前持倉: 數量 {quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            if current_price <= stop_loss_price:
                                logger.info(f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            if current_price >= take_profit_price:
                                logger.info(f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 買入和賣出邏輯
        if latest_row['buy_signal']:
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif latest_row['sell_signal'] and (current_time - latest_row['timestamp']).total_seconds() < 300:  # 5分鐘內
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'] > df['D']) & (df['K'].shift(1) <= df['D'].shift(1))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'] > 30) & (df['RSI'].shift(1) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'] < df['SMA10']) & (df['SMA10'] < df['SMA20']) & (df['SMA20'] < df['SMA50'])

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'] > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系統啟動")
    while True:
        try:
            current_time = datetime.now()

            # 每次循環開始時更新數據
            update_data()

            # 檢查是否在交易時間內
            if current_time.weekday() < 5 and 9 <= current_time.hour < 23:
                logger.info(f"檢查時間: {current_time}")
                for stock in global_STOCK_LIST:
                    logger.info(f"檢查股票: {stock}")
                    check_and_trade(stock)

            # 等待到下一分鐘開始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循環中發生錯誤: {e}", exc_info=True)
            time.sleep(60)  # 發生錯誤時等待1分鐘後繼續

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 1 秒
        time.sleep(1.0)

        # 每 10 個請求後額外等待
        if request_count % 10 == 0:
            time.sleep(1)  # 額外等待 1 秒

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 檢查今日訂單
        try:
            today_orders = trade_ctx.today_orders(status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled])
            stocks_with_open_orders = set(order.symbol for order in today_orders)
            logger.info(f"{current_date}: 今日曾經有訂單的股票: {stocks_with_open_orders}")

            if symbol in stocks_with_open_orders:
                logger.info(f"{current_date}: {symbol} 已有未完成訂單，跳過交易")
                return
        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                position_found = False
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            quantity = position.quantity
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            logger.info(f"{current_date}: {symbol} 當前持倉: 數量 {quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            if current_price <= stop_loss_price:
                                logger.info(f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            if current_price >= take_profit_price:
                                logger.info(f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 買入和賣出邏輯
        if latest_row['buy_signal']:
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif latest_row['sell_signal'] and (current_time - latest_row['timestamp']).total_seconds() < 300:  # 5分鐘內
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'].shift(1) > df['D'].shift(1)) & (df['K'].shift(2) <= df['D'].shift(2))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'].shift(1) > 30) & (df['RSI'].shift(2) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'].shift(1) > df['Signal_Line'].shift(1)) & (df['MACD'].shift(2) <= df['Signal_Line'].shift(2))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'].shift(1) < df['SMA10'].shift(1)) & (df['SMA10'].shift(1) < df['SMA20'].shift(1)) & (df['SMA20'].shift(1) < df['SMA50'].shift(1))

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'].shift(1) > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系統啟動")
    while True:
        try:
            current_time = datetime.now()

            # 每次循環開始時更新數據
            update_data()

            # 檢查是否在交易時間內
            if current_time.weekday() < 5 and 9 <= current_time.hour < 23:
                logger.info(f"檢查時間: {current_time}")
                for stock in global_STOCK_LIST:
                    logger.info(f"檢查股票: {stock}")
                    check_and_trade(stock)

            # 等待到下一分鐘開始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循環中發生錯誤: {e}", exc_info=True)
            time.sleep(60)  # 發生錯誤時等待1分鐘後繼續

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 1 秒
        time.sleep(1.0)

        # 每 10 個請求後額外等待
        if request_count % 10 == 0:
            time.sleep(1)  # 額外等待 1 秒

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.LO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        position_found = False
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            quantity = position.quantity
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            logger.info(f"{current_date}: {symbol} 當前持倉: 數量 {quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            if current_price <= stop_loss_price:
                                logger.info(f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            if current_price >= take_profit_price:
                                logger.info(f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 在检查订单之前设置 current_order_type
        current_order_type = None

        # 如果有持仓且未触发止损或止盈，设置 current_order_type 为 'sell'
        if position_found:
            current_order_type = 'sell'

        # 如果检测到买入信号且没有持仓，设置 current_order_type 为 'buy'
        if latest_row['buy_signal'] and not position_found:
            current_order_type = 'buy'

        # 檢查今日訂單
        try:
            # 获取今天所有未完成或已成交的买入订单
            buy_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Buy
            )

            # 获取今天所有未完成或已成交的卖出订单
            sell_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Sell
            )

            # 提取买入订单和卖出订单涉及的股票符号
            stocks_with_buy_orders = set(order.symbol for order in buy_orders)
            stocks_with_sell_orders = set(order.symbol for order in sell_orders)

            logger.info(f"{current_date}: 今日已有買入訂單的股票: {stocks_with_buy_orders}")
            logger.info(f"{current_date}: 今日已有賣出訂單的股票: {stocks_with_sell_orders}")

            # 如果当前想要执行的交易是买入操作
            if symbol in stocks_with_buy_orders and current_order_type == 'buy':
                logger.info(f"{current_date}: {symbol} 已有買入訂單，跳過買入交易")
                return  # 跳过买入操作

            # 如果当前想要执行的交易是卖出操作
            if symbol in stocks_with_sell_orders and current_order_type == 'sell':
                logger.info(f"{current_date}: {symbol} 已有賣出訂單，跳過賣出交易")
                return  # 跳过卖出操作

        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 买入和卖出逻辑
        if current_order_type == 'buy':
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif current_order_type == 'sell':
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'].shift(1) > df['D'].shift(1)) & (df['K'].shift(2) <= df['D'].shift(2))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'].shift(1) > 30) & (df['RSI'].shift(2) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'].shift(1) > df['Signal_Line'].shift(1)) & (df['MACD'].shift(2) <= df['Signal_Line'].shift(2))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'].shift(1) < df['SMA10'].shift(1)) & (df['SMA10'].shift(1) < df['SMA20'].shift(1)) & (df['SMA20'].shift(1) < df['SMA50'].shift(1))

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'].shift(1) > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系統啟動")
    while True:
        try:
            current_time = datetime.now()

            # 每次循環開始時更新數據
            update_data()

            # 檢查是否在交易時間內
            if current_time.weekday() < 5 and 9 <= current_time.hour < 23:
                logger.info(f"檢查時間: {current_time}")
                for stock in global_STOCK_LIST:
                    logger.info(f"檢查股票: {stock}")
                    check_and_trade(stock)

            # 等待到下一分鐘開始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循環中發生錯誤: {e}", exc_info=True)
            time.sleep(60)  # 發生錯誤時等待1分鐘後繼續

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 1 秒
        time.sleep(1.0)

        # 每 10 個請求後額外等待
        if request_count % 10 == 0:
            time.sleep(1)  # 額外等待 1 秒

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.ELO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.ELO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        # 獲取持倉信息
        position_found = False
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            available_quantity = position.available_quantity  # 使用 available_quantity
                            logger.info(
                                f"{current_date}: {symbol} 當前持倉: 可賣出數量 {available_quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            if current_price <= stop_loss_price:
                                logger.info(
                                    f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), available_quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            if current_price >= take_profit_price:
                                logger.info(
                                    f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), available_quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 在检查订单之前设置 current_order_type
        current_order_type = None

        # 如果有持仓且未触发止损或止盈，设置 current_order_type 为 'sell'
        if position_found:
            current_order_type = 'sell'

        # 如果检测到买入信号且没有持仓，设置 current_order_type 为 'buy'
        if latest_row['buy_signal'] and not position_found:
            current_order_type = 'buy'

        # 檢查今日訂單
        try:
            # 获取今天所有未完成或已成交的买入订单
            buy_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Buy
            )

            # 获取今天所有未完成或已成交的卖出订单
            sell_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Sell
            )

            # 提取买入订单和卖出订单涉及的股票符号
            stocks_with_buy_orders = set(order.symbol for order in buy_orders)
            stocks_with_sell_orders = set(order.symbol for order in sell_orders)

            logger.info(f"{current_date}: 今日已有買入訂單的股票: {stocks_with_buy_orders}")
            logger.info(f"{current_date}: 今日已有賣出訂單的股票: {stocks_with_sell_orders}")

            # 如果当前想要执行的交易是买入操作
            if symbol in stocks_with_buy_orders and current_order_type == 'buy':
                logger.info(f"{current_date}: {symbol} 已有買入訂單，跳過買入交易")
                return  # 跳过买入操作

            # 如果当前想要执行的交易是卖出操作
            if symbol in stocks_with_sell_orders and current_order_type == 'sell':
                logger.info(f"{current_date}: {symbol} 已有賣出訂單，跳過賣出交易")
                return  # 跳过卖出操作

        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 买入和卖出逻辑
        if current_order_type == 'buy':
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif current_order_type == 'sell':
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'].shift(1) > df['D'].shift(1)) & (df['K'].shift(2) <= df['D'].shift(2))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'].shift(1) > 30) & (df['RSI'].shift(2) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'].shift(1) > df['Signal_Line'].shift(1)) & (df['MACD'].shift(2) <= df['Signal_Line'].shift(2))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'].shift(1) < df['SMA10'].shift(1)) & (df['SMA10'].shift(1) < df['SMA20'].shift(1)) & (df['SMA20'].shift(1) < df['SMA50'].shift(1))

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'].shift(1) > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系統啟動")
    while True:
        try:
            current_time = datetime.now()

            # 每次循環開始時更新數據
            update_data()

            # 檢查是否在交易時間內
            if current_time.weekday() < 5 and 9 <= current_time.hour < 23:
                logger.info(f"檢查時間: {current_time}")
                for stock in global_STOCK_LIST:
                    logger.info(f"檢查股票: {stock}")
                    check_and_trade(stock)

            # 等待到下一分鐘開始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循環中發生錯誤: {e}", exc_info=True)
            time.sleep(60)  # 發生錯誤時等待1分鐘後繼續

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等

In [ ]:
import pandas as pd
from longport.openapi import TradeContext, Config, OrderStatus, OrderSide, Market
from config import get_config  # 導入配置函數

def get_today_orders():
    config = get_config()  # 獲取配置
    ctx = TradeContext(config)
    resp = ctx.today_orders()

    # 將響應轉換為DataFrame
    orders_data = []
    for order in resp:
        orders_data.append({
            'order_id': order.order_id,
            'status': str(order.status),
            'stock_name': order.stock_name,
            'quantity': order.quantity,
            'executed_quantity': order.executed_quantity,
            'price': order.price,
            'executed_price': order.executed_price,
            'submitted_at': order.submitted_at,
            'side': str(order.side),
            'symbol': order.symbol,
            'order_type': str(order.order_type),
            'last_done': order.last_done,
            'trigger_price': order.trigger_price,
            'msg': order.msg,
            'tag': str(order.tag),
            'time_in_force': str(order.time_in_force),
            'expire_date': order.expire_date,
            'updated_at': order.updated_at,
            'trigger_at': order.trigger_at,
            'trailing_amount': order.trailing_amount,
            'trailing_percent': order.trailing_percent,
            'limit_offset': order.limit_offset,
            'trigger_status': str(order.trigger_status) if order.trigger_status else None,
            'currency': order.currency,
            'outside_rth': str(order.outside_rth) if order.outside_rth else None,
            'remark': order.remark
        })

    df = pd.DataFrame(orders_data)
    return df


# 設置選項以顯示所有列和行
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 獲取今日訂單
today_orders_df = get_today_orders()

# 打印 DataFrame 的內容
print("DataFrame content:")
print(today_orders_df)

In [ ]:
from longport.openapi import QuoteContext, Period, AdjustType, TradeContext, OrderType, OrderSide, TimeInForceType, OrderStatus
from config import get_config
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime, timedelta
import time
from stock_symbols import STOCK_LIST, STOCK_DICT, ACCOUNT_INFO, calculate_buy_quantity, calculate_sell_quantity
import sys
import io
import logging
from logging.handlers import RotatingFileHandler
import os

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def setup_logging():
    # 創建 logs 目錄（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')

    # 設置日誌格式
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(log_format)

    # 創建 RotatingFileHandler
    file_handler = RotatingFileHandler(
        'logs/trading_system.log', maxBytes=10*1024*1024, backupCount=5, encoding='utf-8'
    )
    file_handler.setFormatter(formatter)

    # 創建 StreamHandler 用於控制台輸出
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    console_handler.setStream(sys.stdout)

    # 獲取根日誌記錄器並設置級別
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 添加處理器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# 在主程序開始時調用此函數
logger = setup_logging()

#####################################################################################

# 獲取配置
try:
    config = get_config()
except Exception as e:
    logger.error(f"獲取配置時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 創建 QuoteContext 和 TradeContext
try:
    quote_ctx = QuoteContext(config)
    trade_ctx = TradeContext(config)
except Exception as e:
    logger.error(f"創建 QuoteContext 或 TradeContext 時發生錯誤: {e}", exc_info=True)
    sys.exit(1)

# 全局變量
global_STOCK_LIST = []
global_STOCK_DICT = {}
global_ACCOUNT_INFO = {}

def update_data():
    global global_STOCK_LIST, global_STOCK_DICT, global_ACCOUNT_INFO
    try:
        # 重新導入模塊以獲取最新數據
        import importlib
        import stock_symbols
        importlib.reload(stock_symbols)
        global_STOCK_LIST = stock_symbols.STOCK_LIST
        global_STOCK_DICT = stock_symbols.STOCK_DICT
        global_ACCOUNT_INFO = stock_symbols.ACCOUNT_INFO
        logger.info(f"數據更新時間: {datetime.now()}")
    except Exception as e:
        logger.error(f"更新數據時發生錯誤: {e}", exc_info=True)

# 獲取蠟燭圖數據
request_count = 0

def get_candlestick_data(symbol):
    global request_count

    try:
        resp = quote_ctx.candlesticks(symbol, Period.Day, 60, AdjustType.NoAdjust)

        # 增加請求計數
        request_count += 1

        # 每次請求後等待 0.1 秒，以控制每秒的總請求數不超過 10 次
        time.sleep(1)

        # 每 10 個請求後，讓系統休息 1 秒，以防止超出限制
        if request_count % 5 == 0:
            time.sleep(1.0)

        # 將數據轉換為 DataFrame
        df = pd.DataFrame([
            {
                'timestamp': candle.timestamp,
                'open': candle.open,
                'high': candle.high,
                'low': candle.low,
                'close': candle.close,
                'volume': candle.volume,
                'turnover': candle.turnover
            }
            for candle in resp
        ])

        # 將 timestamp 列轉換為 datetime 格式
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # 將數值列轉換為適當的數據類型
        numeric_columns = ['open', 'high', 'low', 'close', 'turnover']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

        # 按時間順序排序
        df = df.sort_values('timestamp')

        logger.info(f"成功獲取 {symbol} 的蠟燭圖數據")
        return df

    except longport.OpenApiException as e:
        logger.error(f"處理股票 {symbol} 時發生 OpenApiException 錯誤: {e}")
        return None
    except Exception as e:
        logger.error(f"處理股票 {symbol} 時發生未預期的錯誤: {e}", exc_info=True)
        return None

def process_stocks_in_batches(symbols, batch_size=3):
    """
    将股票列表分批次处理，每批次最多处理 `batch_size` 只股票。
    """
    for i in range(0, len(symbols), batch_size):
        batch = symbols[i:i + batch_size]

        for symbol in batch:
            check_and_trade(symbol)  # 保留原逻辑，执行完整的检查和交易操作

        # 每批次处理完后休息 2 秒，以确保不会超出 API 限制
        time.sleep(2)

#########################################################################################

# 計算KDJ
def calculate_kdj(df, n=9, m1=3, m2=3):
    try:
        low_list = df['low'].rolling(window=n, min_periods=n).min()
        high_list = df['high'].rolling(window=n, min_periods=n).max()
        rsv = (df['close'] - low_list) / (high_list - low_list) * 100
        df['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
        df['D'] = df['K'].ewm(com=m2 - 1, adjust=False).mean()
        df['J'] = 3 * df['K'] - 2 * df['D']
        return df
    except Exception as e:
        logger.error(f"計算KDJ時發生錯誤: {e}", exc_info=True)
        return df

# 計算RSI
def calculate_rsi(df, window=14):
    try:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    except Exception as e:
        logger.error(f"計算RSI時發生錯誤: {e}", exc_info=True)
        return df

# 計算MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    try:
        df['EMA_short'] = df['close'].ewm(span=short_window, adjust=False).mean()
        df['EMA_long'] = df['close'].ewm(span=long_window, adjust=False).mean()
        df['MACD'] = df['EMA_short'] - df['EMA_long']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        return df
    except Exception as e:
        logger.error(f"計算MACD時發生錯誤: {e}", exc_info=True)
        return df

# 計算SMA
def calculate_sma(df, windows=[10, 20, 50]):
    try:
        for window in windows:
            df[f'SMA{window}'] = df['close'].rolling(window=window).mean()
        return df
    except Exception as e:
        logger.error(f"計算SMA時發生錯誤: {e}", exc_info=True)
        return df

#########################################################################################

# 定義下單函數
def place_order(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.ELO,
            OrderSide.Buy,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Auto order from strategy",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

def close_position(symbol, price, quantity):
    try:
        resp = trade_ctx.submit_order(
            symbol,
            OrderType.ELO,
            OrderSide.Sell,
            quantity,
            TimeInForceType.Day,
            submitted_price=Decimal(str(price)),
            remark="Close position",
        )
        logger.info(f"訂單已提交：股票 {symbol}, 價格 {price}, 數量 {quantity}")
        return resp
    except Exception as e:
        logger.error(f"下單失敗：{e}", exc_info=True)
        return None

#########################################################################################

# 主策略邏輯
def run_strategy(df, symbol):
    try:
        latest_row = df.iloc[-1]
        current_time = datetime.now()
        current_date = latest_row['timestamp'].date()
        current_price = latest_row['close']

        # 止損比例和止盈比例
        stop_loss_percentage = Decimal('0.02')
        take_profit_percentage = Decimal('0.02')

        # 獲取持倉信息
        position_found = False
        try:
            positions_response = trade_ctx.stock_positions()
            if positions_response:
                for channel in positions_response.channels:
                    for position in channel.positions:
                        if position.symbol == symbol:
                            cost_price = Decimal(str(position.cost_price))
                            current_price = Decimal(str(current_price))
                            available_quantity = position.available_quantity  # 使用 available_quantity
                            logger.info(f"{current_date}: {symbol} 當前持倉: 可賣出數量 {available_quantity}, 平均成本價 {cost_price}")
                            position_found = True

                            # 檢查止損
                            stop_loss_price = cost_price * (Decimal('1') - stop_loss_percentage)
                            # 止損觸發
                            if current_price <= stop_loss_price:
                                logger.info(
                                    f"{current_date}: {symbol} 觸發止損, 當前價格 {current_price}, 止損價 {stop_loss_price}")
                                order_resp = close_position(symbol, float(current_price), available_quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止損訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止損訂單提交失敗")
                                return  # 執行止損後退出函數

                            # 檢查止盈
                            take_profit_price = cost_price * (Decimal('1') + take_profit_percentage)
                            # 止盈觸發
                            if current_price >= take_profit_price:
                                logger.info(
                                    f"{current_date}: {symbol} 觸發止盈, 當前價格 {current_price}, 止盈價 {take_profit_price}")
                                order_resp = close_position(symbol, float(current_price), available_quantity)
                                if order_resp:
                                    logger.info(f"{current_date}: 止盈訂單提交成功：訂單ID {order_resp.order_id}")
                                else:
                                    logger.warning(f"{current_date}: 止盈訂單提交失敗")
                                return  # 執行止盈後退出函數

                            break
                    if position_found:
                        break
                if not position_found:
                    logger.info(f"{current_date}: {symbol} 當前無持倉")
            else:
                logger.warning(f"{current_date}: 獲取持倉信息失敗")
        except Exception as e:
            logger.error(f"{current_date}: 獲取持倉信息時發生錯誤: {str(e)}")

        # 在检查订单之前设置 current_order_type
        current_order_type = None

        # 如果有持仓且未触发止损或止盈，设置 current_order_type 为 'sell'
        if position_found:
            # 明确检查卖出信号
            if latest_row['sell_signal']:
                current_order_type = 'sell'
                logger.info(f"{current_date}: {symbol} 觸發 RSI > 70 賣出信號")
            else:
                logger.info(f"{current_date}: {symbol} 有持仓，但未检测到卖出信号")

        # 如果检测到买入信号且没有持仓，设置 current_order_type 为 'buy'
        if latest_row['buy_signal'] and not position_found:
            current_order_type = 'buy'
            triggers = []
            if latest_row['kdj_golden_cross']:
                triggers.append("KDJ 黃金交叉")
            if latest_row['rsi_cross_30']:
                triggers.append("RSI 穿過 30")
            if latest_row['macd_golden_cross']:
                triggers.append("MACD 黃金交叉")
            if latest_row['sma_trend']:
                triggers.append("SMA 趨勢")

            logger.info(f"{current_date}: {symbol} 觸發買入信號，觸發的指標: {', '.join(triggers)}")

        # 檢查今日訂單
        try:
            # 获取今天所有未完成或已成交的买入订单
            buy_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Buy
            )

            # 获取今天所有未完成或已成交的卖出订单
            sell_orders = trade_ctx.today_orders(
                status=[OrderStatus.PartialFilled, OrderStatus.New, OrderStatus.NotReported, OrderStatus.WaitToNew, OrderStatus.Filled],
                side=OrderSide.Sell
            )

            # 提取买入订单和卖出订单涉及的股票符号
            stocks_with_buy_orders = set(order.symbol for order in buy_orders)
            stocks_with_sell_orders = set(order.symbol for order in sell_orders)

            logger.info(f"{current_date}: 今日已有買入訂單的股票: {stocks_with_buy_orders}")
            logger.info(f"{current_date}: 今日已有賣出訂單的股票: {stocks_with_sell_orders}")

            # 如果当前想要执行的交易是买入操作
            if symbol in stocks_with_buy_orders and current_order_type == 'buy':
                logger.info(f"{current_date}: {symbol} 已有買入訂單，跳過買入交易")
                return  # 跳过买入操作

            # 如果当前想要执行的交易是卖出操作
            if symbol in stocks_with_sell_orders and current_order_type == 'sell':
                logger.info(f"{current_date}: {symbol} 已有賣出訂單，跳過賣出交易")
                return  # 跳过卖出操作

        except Exception as e:
            logger.error(f"{current_date}: 獲取今日訂單時發生錯誤: {str(e)}")
            return

        # 买入和卖出逻辑
        if current_order_type == 'buy':
            try:
                # 使用 calculate_buy_quantity 函數計算建議買入數量
                quantity = calculate_buy_quantity(symbol, STOCK_DICT, ACCOUNT_INFO)

                if quantity > 0:
                    buy_price = current_price

                    logger.info(f"{current_date}: 檢測到買入信號，準備下單")
                    logger.info(f"建議買入數量: {quantity}")

                    order_resp = place_order(symbol, buy_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 買入訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 買入訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到買入信號，但建議買入數量為0，不執行交易")
            except Exception as e:
                logger.error(f"{current_date}: 執行買入策略時發生錯誤: {str(e)}")

        elif current_order_type == 'sell':
            try:
                # 使用 calculate_sell_quantity 函數計算建議賣出數量
                quantity = calculate_sell_quantity(symbol, STOCK_DICT)

                if quantity > 0:
                    sell_price = current_price

                    logger.info(f"{current_date}: 檢測到賣出信號，準備平倉")
                    logger.info(f"建議賣出數量: {quantity}")

                    order_resp = close_position(symbol, sell_price, quantity)

                    if order_resp:
                        logger.info(f"{current_date}: 賣出訂單提交成功：訂單ID {order_resp.order_id}")
                    else:
                        logger.warning(f"{current_date}: 賣出訂單提交失敗")
                else:
                    logger.info(f"{current_date}: 檢測到賣出信號，但沒有可平倉的持倉")
            except Exception as e:
                logger.error(f"{current_date}: 執行賣出策略時發生錯誤: {str(e)}")

        else:
            logger.info(f"{current_date}: 沒有檢測到有效的交易信號或信號已過期")

    except Exception as e:
        logger.error(f"執行策略時發生未預期的錯誤: {e}", exc_info=True)

# 檢查和交易函數
def check_and_trade(symbol):
    try:
        df = get_candlestick_data(symbol)
        if df is not None:
            df = calculate_kdj(df)
            df = calculate_rsi(df)
            df = calculate_macd(df)
            df = calculate_sma(df)

            # 識別KDJ黃金交叉
            df['kdj_golden_cross'] = (df['K'].shift(1) > df['D'].shift(1)) & (df['K'].shift(2) <= df['D'].shift(2))

            # 識別RSI穿過30的情況
            df['rsi_cross_30'] = (df['RSI'].shift(1) > 30) & (df['RSI'].shift(2) <= 30)

            # 識別MACD黃金交叉
            df['macd_golden_cross'] = (df['MACD'].shift(1) > df['Signal_Line'].shift(1)) & (df['MACD'].shift(2) <= df['Signal_Line'].shift(2))

            # 識別 close < SMA10 < SMA20 < SMA50 的情況
            df['sma_trend'] = (df['close'].shift(1) < df['SMA10'].shift(1)) & (df['SMA10'].shift(1) < df['SMA20'].shift(1)) & (df['SMA20'].shift(1) < df['SMA50'].shift(1))

            # 結合所有買入信號
            df['buy_signal'] = df['rsi_cross_30'] | df['macd_golden_cross'] | df['kdj_golden_cross'] | df['sma_trend']

            # 添加賣出信號（這裡以RSI>70為例，您可以根據需要調整）
            df['sell_signal'] = df['RSI'].shift(1) > 70

            run_strategy(df, symbol)
        else:
            logger.warning(f"無法獲取 {symbol} 的蠟燭圖數據")
    except Exception as e:
        logger.error(f"檢查和交易 {symbol} 時發生錯誤: {e}", exc_info=True)

def main():
    logger.info("交易系统启动")
    while True:
        try:
            current_time = datetime.now()

            # 每次循环开始时更新数据
            update_data()

            if current_time.weekday() < 5 and 9 <= current_time.hour < 23:
                logger.info(f"检查时间: {current_time}")

                # 将 60 只股票分批次处理，每批最多 3 只股票
                process_stocks_in_batches(global_STOCK_LIST, batch_size=3)

            # 等待到下一分钟开始
            next_minute = current_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
            time_to_sleep = (next_minute - current_time).total_seconds()
            time.sleep(time_to_sleep)
        except Exception as e:
            logger.error(f"主循环中发生错误: {e}", exc_info=True)
            time.sleep(60)  # 发生错误时等待1分钟后继续

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"程序發生嚴重錯誤: {e}", exc_info=True)
        # 這裡可以添加其他錯誤處理邏輯，如發送警報郵件等